In [ ]:
import pandas as pd
import numpy as np

import pydsm

from pydsm import qualh5

import h5py
from pydsm import dsm2h5

# Read GTM h5 file from 

In [ ]:
gtm_file = '../tests/data/gtm_sample_output/on_cell/historical_gtm.h5'

In [ ]:
gh5 = h5py.File(gtm_file, 'r+')

In [ ]:
dsm2h5.get_model(gh5) # wrong. should return gtm

List of constituents

In [ ]:
dsm2h5.read_table_as_df(gh5, '/output/constituent_names').iloc[:,0].values.tolist()

Channels with cell informatioin

In [ ]:
channels = dsm2h5.read_table_as_df(gh5, '/geometry/channel')
channels

List of data paths in GTM

In [ ]:
gtm_datapaths = ['/output/'+tname for tname in gh5['/output']]
gtm_datapaths

Connection table

In [ ]:
dsm2h5.read_table_as_df(gh5, '/geometry/connection')

Attributes of Cell concentration table

In [ ]:
attrs = dsm2h5.read_attributes_from_table(gh5['/output/cell concentration'])
attrs

read the entire cell concentrations. slicing by time is also available

In [ ]:
dfcell = dsm2h5.read_time_indexed_table(gh5,'/output/cell concentration',None,0)
dfcell

In [ ]:
import hvplot.pandas
hvplot.extension('bokeh')

In [ ]:
channels.query('channel_num == 441')

In [ ]:
dfcell.loc[:,3613:3617].hvplot(rot=40)

Cell concentration shape is time x constituents x cell_number

The dataframe above shows the attributes for the table
The start time and interval together should help convert the first dimension to time indexed array 
Each constituent can be returned with the cell ids

Segments are where cells are defined for each channel. Segments seem to be grouping of cells within a channel

In [ ]:
dfsegments = dsm2h5.read_table_as_df(gh5, '/geometry/segment')
dfsegments

In [ ]:
dfsegments['nx'].sum()

In [ ]:
import geopandas as gpd

In [ ]:
from shapely.geometry import LineString, Point
import math
def cut(line, distance):
    # Cuts a line in two at a distance from its starting point
    if distance <= 0.0 or math.isclose(distance, line.length, abs_tol=0.01) or distance >= line.length:
        return [LineString(line)]
    coords = list(line.coords)
    for i, p in enumerate(coords):
        pd = line.project(Point(p))
        if pd == distance:
            return [
                LineString(coords[:i+1]),
                LineString(coords[i:])]
        if pd > distance:
            cp = line.interpolate(distance)
            return [
                LineString(coords[:i] + [(cp.x, cp.y)]),
                LineString([(cp.x, cp.y)] + coords[i:])]

def split(line, distance):
    segments = []
    while math.isclose(line.length, distance, abs_tol=0.01) or line.length > distance:
        segline = cut(line, distance)
        if len(segline) == 2:
            segments.append(segline[0])
            line = segline[1]
        elif len(segline) == 1:
            segments.append(segline[0])
            break;
    return segments

In [ ]:
channel_lines = gpd.read_file('../pydsm/maps/v8.2-opendata/gisgridmapv8.2channelcenterlines/dsm2_channels_centerlines_8_2.shp')

In [ ]:
nodes = gpd.read_file('../pydsm/maps/v8.2-opendata/gisgridmapv8.2nodes/dsm2_nodes_8_2.shp')

Are the channel lines specified as lines from upnode to downnode? If not make them so that channel lines are specified as points from upnode to downnode

In [ ]:
import math

In [ ]:
import shapely.wkt
import shapely.ops


def reverse_geom(geom):
    def _reverse(x, y, z=None):
        if z:
            return x[::-1], y[::-1], z[::-1]
        return x[::-1], y[::-1]

    return shapely.ops.transform(_reverse, geom)

In [ ]:
channel_lines = channel_lines.merge(channels, left_on='id', right_on='channel_num')
channel_lines

In [ ]:
new_geom = []
for i,r in channel_lines.iterrows():
    line = r['geometry']
    upnode = nodes.query(f'id=={r["up_node"]}').geometry.values[0]
    dnnode = nodes.query(f'id=={r["down_node"]}').geometry.values[0]
    #print(r["channel_num"],line.project(upnode),line.project(dnnode))
    if line.project(upnode) < line.project(dnnode):
        pass
    else:
        print(f'reversing: r["channel_num"]')
        line = reverse_geom(line)    
    new_geom.append(line)

In [ ]:
cells = []
for i,r in channel_lines.iterrows():#.query('channel_length > 25000').iterrows():
    line = r['geometry']
    nsegments = (r['end_cell']-r['start_cell']+1)
    segs = split(line, line.length/nsegments)
    cells = cells + list(zip(range(r['start_cell'],r['end_cell']+1), segs, [r['channel_num']]*nsegments))

In [ ]:
cell_gis = pd.DataFrame(cells)
cell_gis.columns=['id','geometry','channel_num']
cell_gis = gpd.GeoDataFrame(cell_gis, geometry='geometry', crs=channel_lines.crs)
cell_gis.plot()

In [ ]:
import holoviews as hv
from holoviews import opts, dim
import geoviews as gv
import cartopy.crs as ccrs
hv.extension('bokeh')
import hvplot
#
import panel as pn
pn.extension()
#
import warnings
warnings.filterwarnings('ignore')


In [ ]:
date='2016-05-17 20:00'
cell_vals = dfcell.loc[pd.to_datetime(date)]
cell_vals.name = 'ec'
mcell_gis = cell_gis.join(cell_vals, on='id')
mcell_gis

In [ ]:
tmap=hv.element.tiles.CartoLight().opts(width=600,height=500,alpha=0.5)

In [ ]:
start_time = dfcell.index[0]

In [ ]:
def get_data(ti):
    df = dfcell.iloc[ti]
    df.name='ec'
    return df

def channel_map(ti):
    cell_vals=get_data(ti)
    cur_time = start_time+i*dfcell.index.freq# ti*pd.to_timedelta('1D')#
    tlabel = cur_time.strftime('%Y-%m-%d %H:%M')
    mcell_gis=cell_gis.join(cell_vals, on='id')
    chans_plot=gv.Path(mcell_gis, vdims=['ec'], 
                       crs=ccrs.UTM(10)).opts(cmap='rainbow4',
                                              color='ec', clim=(300,800), colorbar=True, 
                                              line_width=2, framewise=False)
    return chans_plot.opts(title='Time: %s'%tlabel,framewise=False)

time_index = pn.widgets.IntSlider(start=0,end=len(dfcell))
dmap=hv.DynamicMap(channel_map, streams = {'ti':time_index})
pn.Column(time_index, tmap*dmap)

In [ ]:
view = channel_map(0)
from bokeh.io import push_notebook, show, output_notebook
rendered_view=hv.render(tmap*view)
pr=rendered_view.renderers[1]
target = show(rendered_view, notebook_handle=True)
# faster to update view?
def update_view(ti=0):
    cell_vals=get_data(ti)
    cur_time = start_time+ti*dfcell.index.freq# ti*pd.to_timedelta('1D')#
    tlabel = cur_time.strftime('%Y-%m-%d %H:%M')
    rendered_view.title.text=tlabel
    #
    mcell_gis=cell_gis.join(cell_vals,on='id')
    pr.data_source.data['color']=mcell_gis.ec.values
    #pr.data_source.data['ec']=mcell_gis.ec.values
    push_notebook(handle=target)
#
pn.interact(update_view,ti=(0,len(dfcell)))